In [103]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import csv
from IPython.display import clear_output

In [104]:
data_path = './data'

In [105]:
train = pd.read_csv(data_path + '/train.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
y_m_train = train['label'].values
X_m_train = train.drop(['label'], axis=1)

In [108]:
X_m_train = X_m_train.values.reshape(-1, 28, 28, 1)

In [109]:
X_m_train, X_valid, y_m_train, y_valid = train_test_split(X_m_train, y_m_train, test_size=0.2, random_state=42, stratify=y_m_train)

In [110]:
import pickle
def de_pickler(pickle_path):
    with open(pickle_path, 'rb') as f:
        return pickle.load(f)

pickle_path = './data/MNIST-120k'
qmnist = de_pickler(pickle_path)

q_data = qmnist['data']
q_labels = qmnist['labels']
print(q_data.shape, q_labels.shape)

(120000, 28, 28) (120000, 1)


In [111]:
X_q_train = np.expand_dims(q_data, axis=-1)
y_q_train = np.squeeze(q_labels)

In [112]:
print(X_q_train.shape, y_q_train.shape)
print(X_m_train.shape, y_m_train.shape)

(120000, 28, 28, 1) (120000,)
(33600, 28, 28, 1) (33600,)


In [113]:
X_train = np.concatenate((X_m_train, X_q_train), axis=0)


X_train = X_train / 255.0
X_valid = X_valid / 255.0

In [114]:
y_train = np.concatenate((y_m_train, y_q_train), axis=0)

y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)

In [96]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1)

datagen.fit(X_train)

In [97]:

cnn_model = Sequential([
    Conv2D(64, 3, padding='same', activation='relu', input_shape=X_train[0].shape),
    BatchNormalization(),
    MaxPool2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D(),
    Flatten(),
    Dense(512, 'relu'),
    Dense(256, 'relu'),
    Dense(10, 'softmax')
])

cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_4 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1568)             

In [98]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)

cnn_history = cnn_model.fit(X_train, y_train, epochs=100,
                            validation_data=(X_valid, y_valid),
                            callbacks=[early_stopping],
                            verbose=2)

clear_output()

In [ ]:
test = pd.read_csv(data_path + './test.csv')
test = test.values.reshape(-1, 28, 28, 1)
test = test / 255.0

In [99]:
submission_predictions = cnn_model.predict(test)

In [100]:
submission_predictions = np.argmax(submission_predictions, axis = 1)

In [102]:
print(submission_predictions)

[2 2 9 ... 3 9 2]


In [101]:
with open('./submission_qmnist.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['ImageId', 'Label'])
    for i, item in enumerate(submission_predictions):
      spamwriter.writerow([i+1, item])